Devkota : Transformer that can write a poem 

In [ ]:
# imports and setup
import torch #tensors
import torch.nn as nn  # layers and activatiom
from torch.nn import functional as F  # softmax and cross_entropy

In [2]:
# hyper parameters 

# processes 16 text sequences at once
batch_size = 16 
#  maximum content length 
block_size = 32
# num of training steps : model will see 5000 batches of data 
max_iters = 5000
# check progress every 100 steps 
eval_interval = 100
# how big learning steps are 1e-3 = 0.001 
# too high = unstable, too low = slow learning 
learning_rate = 1e-3

In [5]:
# checking for gpu 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
# avg loss over 200 batches when evaluating
eval_iters = 200
# embedding dimesnion
n_embd = 64 
# num of attention heads 
n_head = 4 
# num of transformer bloacks 
n_layer = 4 
dropout = 0.0

In [7]:
# random seed for reproduceblity 
torch.manual_seed(1337)

In [ ]:
# loading and preparing data 
with open('poem.txt','r', encoding='utf-8') as f:
    text = f.read() #reads from the file 

In [13]:
# create vocab of unique characters 
# set(text) : get all unique characters 
# list() : convert to list 
# sorted : sorts alphabetically 
# eg if text = "hello" chrs = ['e','h','l','o']
chars = sorted(list(set(text)))

In [17]:
# gives the lenght of chars, total element
vocab_size = len(chars)

In [ ]:
# string to integer 
stoi = {}
i = 0 
for ch in chars:
    stoi[ch] = i 
    i +=1

# or 
# stoi = {ch:i, for i,ch in enumerate(chars)}

In [20]:
# integer to string 
itos  = {}
i = 0 
for ch in chars:
    itos[i] = ch 
    i +=1


# or 
# stoi = {i:ch, for i,ch in enumerate(chars)}


In [24]:
# frrom string to number 
encode = lambda s: [stoi[c] for c in s]
encode("नमस्कार")


[41, 46, 53, 65, 22, 55, 48]

In [26]:
decode = lambda s:[itos[c] for c in s]
decode([41, 46, 53, 65, 22, 55, 48])

['न', 'म', 'स', '्', 'क', 'ा', 'र']

In [27]:
# train val split 
data = torch.tensor(encode(text),dtype=torch.long)

In [30]:
# 90% data into n 
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [37]:
# data loading function 
def get_batch(split):
    if split == 'train':
        data = train_data
    else:
        data = val_data
    # generate batch_size random starting psitions
    ix = torch.randint(len(data) - block_size, (batch_size))

    # create lsits to store the sequences 
    x_list = []
    y_list = []

    # manually loop through each random index 
    for start in ix:
        start = int(start) # ensure int 

        # input sequence
        x_seq = data[start : start + block_size]
        x_list.append(x_seq)

        # target sequence (shifeted by 1)
        y_seq = data[start + 1: start + block_size + 1]
        y_list.append(y_seq)

    x = torch.stack(x_list, dim=0)
    y = torch.stack(y_list, dim=0)
    x = x.to(device)
    y = t.to(device)


    return x,y

In [38]:
# loss estimation function 
@torch.no_grad() # disables gradient calcualtion 
def estimate_loss():
    out = {}
    model.eval() #puts model in evaluation mode
    for split in ['train','val']:  # loop through train and val sets
        losses = torch.zeros(eval_iters)
        for k in rage(eval_iters): 
            X, Y = get_batch(split) # get a batch 
            logits, loss = model(X,Y) # run through model, 
            losses[k] = loss.item()  #store the loss , .item()- convert to python number
            out[split] = losses.mean() #average all losses 
            model.train()
            return out 



In [42]:
# single attention head 
class Head(nn.Module):

    def __init__(self,head_size):
        super().__init__()

        # three linear transformations 
        # key : "what information do I have"
        # query : What info I am looking for
        # value : "what is my actual content"
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        # lower traingular matrix fofr casual masking 
        '''[[1,0,0,0],
         [1,1,0,0],
         [1,1,1,0],
         [1,1,1,1],
         ]
         prevent looking at future tokens: token n can only see n and all n-x tokens 
         '''
        #  registerbuffer ; saves this as a part f the model but doesn't train it 
        self.dropout = nn.Dropout(dropout)

        def forward(self,x):
            B,T,C = x.shape  # B : batch_size(16), T : time/sequence = 32, channedl/embed dimesn : 64
            k = self.key(x)
            q = self.query(x)
            # q @ l.transpose(-2,-1): Matrix multiply queries with keys
            # q.shape : (B,T, head_size)
            # result: (B,T,T) - every token's query looks at every token's key 
            # * C** -0.5 : scale by 1/root(head_size) to prevent values from getting too large
            wei = q @ k.transpose(-2,-1) * C** - 0.5
             # wwhere tril is 0(future position),set attention to -infinity 
            # after softmax, -infinity becomes 0, effectively blocking future information 
            wei = wei.masked_fil(self.tril[:T,:T] == 0, float('-inf'))
            # convert to probablities 0 to 1 
            wei = F.softmax(wei, dim=-1) # (B,T,T)
            # apply dropout to attention weights 
            wei = self.dropout(wei)
            # transform x to values 
            # multiply attentio weights by values 
            # each token gets a weighted sum of all previous tokens' values 
            v = self.value(x)
            out = wei @ v 
            return out 

In [ ]:
# Multi-head attention 
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList()  # create an empty ModuleList
        for _ in range(num_heads):    # loop num_heads times
            head = Head(head_size)    # create a Head module 
            self.heads.append(head)   # add it to the ModuleList
        self.proj = nn.Linear(n_embd, n_embd) # combines output 
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # run all heads and concatenate 
        #  each head outputs(B,T,head_size)
        # cat alog last ddimension(B,T,num_heads * head_size)
        # since num_heads * head_size = n_embd, output is (bmT,n_emb)
        out = torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out))
        return out 

In [49]:
# feed-forward network
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4* n_embd),
            nn.ReLU(),
            nn.Linear(4* n_embd,n_embd),
            nn.Dropout(dropout),
        )
    def forward(self,x):
        return self.net(x)

# two-layerd neural network 
# 1. expad 64-> 256 dim 
# 2. Relu activation :; max(0,x) 
# 3. Compress : 256->64 dim 
# 4.dropout


In [ ]:
# transformer block 

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd